In [1]:
import Ipynb_importer
import glob
from PIL import Image
from getCifarImg2Disk import LOAD_CIFAR_LABELS
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

importing Jupyter notebook from getCifarImg2Disk.ipynb


In [2]:
"""
获取标签集
"""
root_path = './data/cifar-10-batches-py/'
file_name = 'batches.meta'
label_name = LOAD_CIFAR_LABELS(root_path + file_name)
label_idx = [id for id in range(len(label_name))]
label_dict = dict(zip(label_name, label_idx))

In [3]:
"""
定义myDataset类
"""
def default_loader(path):
    return Image.open(path).convert("RGB") #将PIL转为RGB格式

class myDataset(Dataset):
    def __init__(self, img_list, transform = None, loader = default_loader):
        super(myDataset, self).__init__()
        imgPaths = [] # 存储所有的图片路径
        labelIndice = [] #存储全部的标签索引


        for img_item in img_list:
            img_label_name = img_item.split("/")[-2] #获取图片标签名称
            """
            注意：python路径中分隔符默认为"/"，window系统中路径中分隔符为“\”，window系统下 glob.glob（）返回的文件路径为"\\"，并将“\”转变为“\\”,不会改变路径中的"/"
            """
            imgPaths.append(img_item)
            labelIndice.append(label_dict[img_label_name])

        self.imgPaths = imgPaths
        self.labelIndice = labelIndice
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        img_path = self.imgPaths[index]
        img_label = self.labelIndice[index]
        img_data = self.loader(img_path)
        if self.transform is not None:
            img_data = self.transform(img_data)
        return img_data, img_label

    def __len__(self):
        return len(self.imgPaths)